In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

pair = "EUR_USD"
# parquet_path = Path("/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h") / f"{pair}.parquet"
# parquet_path = Path("/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h") / f"{pair}.parquet"
norm_robust_path = Path('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/eur_norm_robut.parquet')
eur_standard = Path('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD.parquet')
df = pd.read_parquet(eur_standard)

dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)



saving_path = f'./logs/24nov/not_norm_flat/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = ForexTradingEnv(
        df=train_df,
        pair='EUR_USD',

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair='EUR_USD',
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=100_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}tensorboard/',
)

model.learn(
    total_timesteps=5_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}best_model.zip')
train_env.save(f'{saving_path}vec_normalize.pkl')